In [1]:
from concrete_autoencoder import ConcreteAutoencoderFeatureSelector
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, LeakyReLU
from keras import backend as K
import numpy as np


In [2]:
import tensorflow as tf

In [3]:
import torch


from torch import nn
from torch.autograd import Variable
from torch.nn import functional as F

import numpy as np

import matplotlib.pyplot as plt
#from sklearn.manifold import TSNE

#import math

#import gc

from utils import *

from sklearn.preprocessing import MinMaxScaler

from scipy.stats import pearsonr

import seaborn as sns

In [4]:
cuda = True if torch.cuda.is_available() else False

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

device = torch.device("cuda:0" if cuda else "cpu")
#device = 'cpu'
print("Device")
print(device)

Device
cuda:0


In [5]:
D = 30
N = 10000
z_size = 8

In [6]:
train_data, test_data = generate_synthetic_data_with_noise(N, z_size, D)

In [ ]:
class CEConcreteAutoencoderFeatureSelector(ConcreteAutoencoderFeatureSelector):
    def __init__(self, K, output_function, num_epochs = 300, batch_size = None, learning_rate = 0.001, start_temp = 10.0, min_temp = 0.1, tryout_limit = 5):
            super(CEConcreteAutoencoderFeatureSelector, self).__init__(K, output_function, num_epochs, batch_size, learning_rate, start_temp, min_temp, tryout_limit)
    
    
    def fit(self, X, Y = None, val_X = None, val_Y = None):
        if Y is None:
            Y = X
        assert len(X) == len(Y)
        validation_data = None
        if val_X is not None and val_Y is not None:
            assert len(val_X) == len(val_Y)
            validation_data = (val_X, val_Y)
        
        if self.batch_size is None:
            self.batch_size = max(len(X) // 256, 16)
        
        num_epochs = self.num_epochs
        steps_per_epoch = (len(X) + self.batch_size - 1) // self.batch_size
        
        for i in range(self.tryout_limit):
            
            K.set_learning_phase(1)
            
            inputs = Input(shape = X.shape[1:])

            alpha = math.exp(math.log(self.min_temp / self.start_temp) / (num_epochs * steps_per_epoch))
            
            self.concrete_select = ConcreteSelect(self.K, self.start_temp, self.min_temp, alpha, name = 'concrete_select')

            selected_features = self.concrete_select(inputs)

            outputs = self.output_function(selected_features)

            self.model = Model(inputs, outputs)

            self.model.compile(Adam(self.learning_rate), loss = 'binary_crossentropy')
            
            print(self.model.summary())
            
            stopper_callback = StopperCallback()
            
            hist = self.model.fit(X, Y, self.batch_size, num_epochs, verbose = 1, callbacks = [stopper_callback], validation_data = validation_data)#, validation_freq = 10)
            
            if K.get_value(K.mean(K.max(K.softmax(self.concrete_select.logits, axis = -1)))) >= stopper_callback.mean_max_target:
                break
            
            num_epochs *= 2
        
        self.probabilities = K.get_value(K.softmax(self.model.get_layer('concrete_select').logits))
        self.indices = K.get_value(K.argmax(self.model.get_layer('concrete_select').logits))
            
        return self

convert torch tensors to tf tensors

In [7]:
def tf_to_to(df):
    df = df.cpu().numpy()
    return tf.convert_to_tensor(df)

In [8]:
train_data = tf_to_to(train_data)
test_data = tf_to_to(test_data)

In [9]:
def decoder(x):
    x = Dense(D)(x)
    x = LeakyReLU(0.2)(x)
    x = Dense(2*D)(x)
    x = tf.keras.activations.sigmoid(x)
    return x

selector = CEConcreteAutoencoderFeatureSelector(K = 3*z_size, output_function = decoder, num_epochs = 800)

selector.fit(train_data, train_data, test_data, test_data)

Instructions for updating:
Simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 60)]              0         
_________________________________________________________________
concrete_select (ConcreteSel (None, 24)                1441      
_________________________________________________________________
dense (Dense)                (None, 30)                750       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 30)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 60)                1860      
_________________________________________________________________
tf_op_layer_Sigmoid (TensorF [(None, 60)]           

mean max of probabilities: 0.57132477 - temperature 7.4427657
Epoch 42/800
259/259 [==============================] - 0s 1ms/step - loss: 0.0145 - val_loss: 0.0818
mean max of probabilities: 0.57639974 - temperature 7.389338
Epoch 43/800
259/259 [==============================] - 0s 1ms/step - loss: 0.0143 - val_loss: 0.0833
mean max of probabilities: 0.579287 - temperature 7.3363075
Epoch 44/800
259/259 [==============================] - 0s 1ms/step - loss: 0.0142 - val_loss: 0.0843
mean max of probabilities: 0.58025134 - temperature 7.283646
Epoch 45/800
259/259 [==============================] - 0s 1ms/step - loss: 0.0141 - val_loss: 0.0845
mean max of probabilities: 0.5792597 - temperature 7.231372
Epoch 46/800
259/259 [==============================] - 0s 1ms/step - loss: 0.0140 - val_loss: 0.0857
mean max of probabilities: 0.57708335 - temperature 7.179467
Epoch 47/800
259/259 [==============================] - 0s 1ms/step - loss: 0.0139 - val_loss: 0.0838
mean max of probabiliti

259/259 [==============================] - 0s 1ms/step - loss: 0.0110 - val_loss: 0.0816
mean max of probabilities: 0.69083667 - temperature 5.154489
Epoch 93/800
259/259 [==============================] - 0s 1ms/step - loss: 0.0110 - val_loss: 0.0772
mean max of probabilities: 0.69143724 - temperature 5.1174903
Epoch 94/800
259/259 [==============================] - 0s 1ms/step - loss: 0.0110 - val_loss: 0.0770
mean max of probabilities: 0.6910565 - temperature 5.0807657
Epoch 95/800
259/259 [==============================] - 0s 1ms/step - loss: 0.0110 - val_loss: 0.0763
mean max of probabilities: 0.69135696 - temperature 5.044297
Epoch 96/800
259/259 [==============================] - 0s 1ms/step - loss: 0.0110 - val_loss: 0.0773
mean max of probabilities: 0.6910593 - temperature 5.008082
Epoch 97/800
259/259 [==============================] - 0s 1ms/step - loss: 0.0109 - val_loss: 0.0759
mean max of probabilities: 0.6926916 - temperature 4.972146
Epoch 98/800
259/259 [==============

259/259 [==============================] - 0s 1ms/step - loss: 0.0102 - val_loss: 0.0373
mean max of probabilities: 0.67068213 - temperature 3.5955427
Epoch 143/800
259/259 [==============================] - 0s 1ms/step - loss: 0.0101 - val_loss: 0.0363
mean max of probabilities: 0.6698408 - temperature 3.5697367
Epoch 144/800
259/259 [==============================] - 0s 1ms/step - loss: 0.0101 - val_loss: 0.0367
mean max of probabilities: 0.6693678 - temperature 3.544111
Epoch 145/800
259/259 [==============================] - 0s 1ms/step - loss: 0.0101 - val_loss: 0.0361
mean max of probabilities: 0.6683192 - temperature 3.5186768
Epoch 146/800
259/259 [==============================] - 0s 1ms/step - loss: 0.0101 - val_loss: 0.0351
mean max of probabilities: 0.666984 - temperature 3.4934206
Epoch 147/800
259/259 [==============================] - 0s 1ms/step - loss: 0.0101 - val_loss: 0.0350
mean max of probabilities: 0.6669879 - temperature 3.4683442
Epoch 148/800
259/259 [========

259/259 [==============================] - 0s 1ms/step - loss: 0.0096 - val_loss: 0.0177
mean max of probabilities: 0.6827653 - temperature 2.5080981
Epoch 193/800
259/259 [==============================] - 0s 988us/step - loss: 0.0096 - val_loss: 0.0166
mean max of probabilities: 0.6870813 - temperature 2.4900954
Epoch 194/800
259/259 [==============================] - 0s 1ms/step - loss: 0.0096 - val_loss: 0.0167
mean max of probabilities: 0.6912076 - temperature 2.4722188
Epoch 195/800
259/259 [==============================] - 0s 1ms/step - loss: 0.0096 - val_loss: 0.0164
mean max of probabilities: 0.6964161 - temperature 2.454479
Epoch 196/800
259/259 [==============================] - 0s 1ms/step - loss: 0.0095 - val_loss: 0.0163
mean max of probabilities: 0.70133895 - temperature 2.436861
Epoch 197/800
259/259 [==============================] - 0s 994us/step - loss: 0.0095 - val_loss: 0.0163
mean max of probabilities: 0.7064433 - temperature 2.419368
Epoch 198/800
259/259 [=====

...

259/259 [==============================] - 0s 1ms/step - loss: 0.0076 - val_loss: 0.0076
mean max of probabilities: 0.99983716 - temperature 0.2044824
Epoch 541/800
259/259 [==============================] - 0s 945us/step - loss: 0.0076 - val_loss: 0.0076
mean max of probabilities: 0.9998382 - temperature 0.20301473
Epoch 542/800
259/259 [==============================] - 0s 1ms/step - loss: 0.0076 - val_loss: 0.0076
mean max of probabilities: 0.9998382 - temperature 0.2015575
Epoch 543/800
259/259 [==============================] - 0s 927us/step - loss: 0.0076 - val_loss: 0.0076
mean max of probabilities: 0.9998388 - temperature 0.20011069
Epoch 544/800
259/259 [==============================] - 0s 963us/step - loss: 0.0076 - val_loss: 0.0076
mean max of probabilities: 0.9998391 - temperature 0.1986744
Epoch 545/800
259/259 [==============================] - 0s 967us/step - loss: 0.0076 - val_loss: 0.0076
mean max of probabilities: 0.99984 - temperature 0.1972485
Epoch 546/800
259/259

mean max of probabilities: 0.9998541 - temperature 0.14366925
Epoch 590/800
259/259 [==============================] - 0s 943us/step - loss: 0.0076 - val_loss: 0.0076
mean max of probabilities: 0.99985415 - temperature 0.1426382
Epoch 591/800
259/259 [==============================] - 0s 992us/step - loss: 0.0076 - val_loss: 0.0076
mean max of probabilities: 0.9998548 - temperature 0.14161435
Epoch 592/800
259/259 [==============================] - 0s 948us/step - loss: 0.0076 - val_loss: 0.0076
mean max of probabilities: 0.99985534 - temperature 0.14059795
Epoch 593/800
259/259 [==============================] - 0s 955us/step - loss: 0.0076 - val_loss: 0.0076
mean max of probabilities: 0.9998555 - temperature 0.13958877
Epoch 594/800
259/259 [==============================] - 0s 1ms/step - loss: 0.0076 - val_loss: 0.0076
mean max of probabilities: 0.9998555 - temperature 0.13858691
Epoch 595/800
259/259 [==============================] - 0s 1ms/step - loss: 0.0076 - val_loss: 0.0076
m

259/259 [==============================] - 0s 1ms/step - loss: 0.0075 - val_loss: 0.0076
mean max of probabilities: 0.99986815 - temperature 0.100217365
Epoch 640/800
259/259 [==============================] - 0s 1ms/step - loss: 0.0076 - val_loss: 0.0076
mean max of probabilities: 0.9998682 - temperature 0.1
Epoch 641/800
259/259 [==============================] - 0s 1ms/step - loss: 0.0075 - val_loss: 0.0076
mean max of probabilities: 0.9998682 - temperature 0.1
Epoch 642/800
259/259 [==============================] - 0s 1ms/step - loss: 0.0076 - val_loss: 0.0076
mean max of probabilities: 0.9998682 - temperature 0.1
Epoch 643/800
259/259 [==============================] - 0s 1ms/step - loss: 0.0076 - val_loss: 0.0076
mean max of probabilities: 0.9998682 - temperature 0.1
Epoch 644/800
259/259 [==============================] - 0s 1ms/step - loss: 0.0075 - val_loss: 0.0076
mean max of probabilities: 0.9998682 - temperature 0.1
Epoch 645/800
259/259 [==============================] - 

mean max of probabilities: 0.9998765 - temperature 0.1
Epoch 691/800
259/259 [==============================] - 0s 1ms/step - loss: 0.0076 - val_loss: 0.0076
mean max of probabilities: 0.9998773 - temperature 0.1
Epoch 692/800
259/259 [==============================] - 0s 1ms/step - loss: 0.0075 - val_loss: 0.0076
mean max of probabilities: 0.9998781 - temperature 0.1
Epoch 693/800
259/259 [==============================] - 0s 1ms/step - loss: 0.0075 - val_loss: 0.0076
mean max of probabilities: 0.9998784 - temperature 0.1
Epoch 694/800
259/259 [==============================] - 0s 1ms/step - loss: 0.0075 - val_loss: 0.0076
mean max of probabilities: 0.9998784 - temperature 0.1
Epoch 695/800
259/259 [==============================] - 0s 1ms/step - loss: 0.0075 - val_loss: 0.0076
mean max of probabilities: 0.9998791 - temperature 0.1
Epoch 696/800
259/259 [==============================] - 0s 1ms/step - loss: 0.0076 - val_loss: 0.0076
mean max of probabilities: 0.99987894 - temperature 

259/259 [==============================] - 0s 1ms/step - loss: 0.0075 - val_loss: 0.0076
mean max of probabilities: 0.9998905 - temperature 0.1
Epoch 795/800
259/259 [==============================] - 0s 1ms/step - loss: 0.0075 - val_loss: 0.0076
mean max of probabilities: 0.9998901 - temperature 0.1
Epoch 796/800
259/259 [==============================] - 0s 1ms/step - loss: 0.0075 - val_loss: 0.0076
mean max of probabilities: 0.9998901 - temperature 0.1
Epoch 797/800
259/259 [==============================] - 0s 1ms/step - loss: 0.0075 - val_loss: 0.0076
mean max of probabilities: 0.9998903 - temperature 0.1
Epoch 798/800
259/259 [==============================] - 0s 1ms/step - loss: 0.0075 - val_loss: 0.0076
mean max of probabilities: 0.99989027 - temperature 0.1
Epoch 799/800
259/259 [==============================] - 0s 1ms/step - loss: 0.0075 - val_loss: 0.0076
mean max of probabilities: 0.99988985 - temperature 0.1
Epoch 800/800
259/259 [==============================] - 0s 1ms/

In [10]:
ind = selector.get_support(indices=True)

In [11]:
sum(ind < 30)

18

In [12]:
sum(ind >= 30)

6

In [13]:
ind

array([27, 51, 23, 12, 59,  5,  5, 34, 24,  8, 10, 59,  9, 21,  2, 26, 22,
       15, 19, 29,  6, 18, 56, 31])

In [17]:
K.argmax(selector.model.get_layer('concrete_select').logits)

<tf.Tensor: shape=(24,), dtype=int64, numpy=
array([27, 51, 23, 12, 59,  5,  5, 34, 24,  8, 10, 59,  9, 21,  2, 26, 22,
       15, 19, 29,  6, 18, 56, 31])>

In [15]:
selector.model.get_layer('concrete_select').logits.shape

TensorShape([24, 60])